# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

import time 


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
Vacation_data= "../VacationPY/output_data/cities.csv"
vacationData=pd.read_csv(Vacation_data)

vacationData.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Use the Lat and Lng as locations and Humidity as the weight.

locations = vacationData[["Lat", "Lng"]]#.astype(float)

#humidity
humidity = vacationData["Humidity"]#.astype(float)

#dimensions for graph
#figure_layout = {
     #'width' : '1000px',
    # 'height':'400px',
   #  'border':'1px solid black',
  #   'padding':'0px',
 #    'margin':'0 auto 0 auto'
#}

#fig = gmaps.figure(layout=figure_layout)

#fig = gmaps.figure()

#heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
  #                                 dissipating=False, max_intensity=50,
 #                                  point_radius=1)
#fig.add_layer(heatmap_layer)

#fig

fig = gmaps.figure(center=(25.0, 10.0), zoom_level = 1.8)

#fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
temperatureCriteria = vacationData['Max Temp'] < 80
temperatureCriteria = vacationData['Max Temp'] > 70
windSpeedCriteria = vacationData['Wind Speed'] < 10
cloudinessCriteria = vacationData['Cloudiness'] == 0
Wholecriteria = temperatureCriteria & windSpeedCriteria & windSpeedCriteria 

# Use boolean indexing to filter the weather_df dataframe
weather_df = vacationData[Wholecriteria]
weather_df = weather_df.dropna()
weather_df = weather_df.reset_index()
weather_df.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
1,20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
2,23,23,maragogi,97,BR,1558378610,69,-9.01,-35.22,83.94,9.42
3,25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93
4,26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#hotel_df = pd.DataFrame(weather_df, columns=['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Hotel Name'])
#hotel_df

hotel_df = pd.DataFrame(data=vacationData)
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [7]:
#Add a "Hotel Name" column to the DataFrame.

hotel_df.loc[:,'Hotel Name'] = np.nan
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16,NaN
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41,NaN
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37,NaN
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47,NaN
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05,NaN


In [8]:
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(vacationData["City"])):

    lat = vacationData.loc[city]["Lat"]
    lng = vacationData.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
vacationData["Nearest Hotel"] = hotels
vacationData

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Nearest Hotel
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16,NaN,Nearest hotel not found
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41,NaN,Shandrani Beachcomber Resort & Spa
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37,NaN,Qaanaaq Hotel
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47,NaN,Nearest hotel not found
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05,NaN,Southern Sun Waterfront Cape Town
...,...,...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05,NaN,صالون مها حيدر - Salon Maha
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11,NaN,Best Western Plus Plaza Vizcaya Durango
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05,NaN,Le Manoir aux Vaches
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68,NaN,Nearest hotel not found


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map


markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig
#figure_layout = {
    #'width': '900px',
    #'height': '600px',
    #'border': '1px solid black',
    #'padding': '1px',
    #'margin': '0 auto 0 auto'
#}
#fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
#hotel_layer = gmaps.marker_layer(
 #   locations,info_box_content=[info_box_template.format(row) for index, row in hotel_df.iterrows()]
#)


# Add layer
#fig.add_layer(heatmap_layer)
#fig.add_layer(hotel_layer)
# Display figure
#fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
"outputdata/Heatmap.png"